### **Document Answering with RAG and OpenAI API**
---

      1.   Cleaning Resources and Generating Embeddings:

In [ ]:
from openai import OpenAI
import numpy as np

client = OpenAI(api_key="<api_key>")

query = "Who is Lionel Messi?"

response = client.embeddings.create(
    input=query,
    model="text-embedding-3-small"
).data[0].embedding

resource = open("messi.txt", 'r')
resource_list = resource.readlines()
resource.close()

filtered_resources = []
for line in resource_list:
    line = line.strip()
    if line != "" and len(line) >= 100:
        filtered_resources.append(line)

resource_responses = client.embeddings.create(
    input=filtered_resources,
    model="text-embedding-3-small"
).data

resource_embeddings = {}
for i in range(len(resource_responses)):
    resource_embeddings[i] = np.array(resource_responses[i].embedding)

query_embedding = np.array(response)

      2.   Calculating Simularity and finding most compatible resource:

In [ ]:
def cosine_similarity(A, B):
    return np.dot(A,B) / (np.linalg.norm(A) * np.linalg.norm(B))

similarity = {}
for i in range(len(resource_embeddings)):
    similarity[i] = cosine_similarity(query_embedding, resource_embeddings[i])

most_similar_resource_index = max(similarity, key=similarity.get)
most_similar_resource = filtered_resources[most_similar_resource_index]

print(f"Resource with highest similarity: {most_similar_resource}")
print(f"Similarity Score: {similarity[most_similar_resource_index]}")

Resource with highest similarity: Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists

      3.   Creating prompt and asking LLM for answer:

In [ ]:
prompt = f"""Based on the following resource, answer the question: {query}

Resource: {most_similar_resource}
"""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4",
)

print("LLM Response:")
print(chat_completion.choices[0].message.content.strip())

LLM Response:
Lionel Messi, also known as Leo Messi, is an Argentine professional footballer born on 24 June 1987. He is widely regarded as one of the greatest players of all time and has set numerous records throughout his career. He plays as a forward for Major League Soccer club Inter Miami and the Argentina national team, acting as captain for both. Messi has a plethora of individual accolades, including eight Ballon d'Or awards and eight FIFA World Player of the Year awards. He is the most decorated player in professional football history, with 45 team trophies including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi is also a prolific goalscorer and creative playmaker, having scored over 850 senior career goals for club and country.
